# Imports

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVR, SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

rng = np.random.RandomState(42)

# Data

In [2]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

print(data.shape)
data.head()

(40778, 54)


,DATE,TARGET,TMAX_d1,TMAX_d1_d2_diff,TMAX_3day_mean,TMAX_3day_std,TMAX_3day_min,TMAX_3day_max,TMAX_3day_median,TMAX_5day_mean,...,MONTH_11,MONTH_12,STATION_USW00003967,STATION_USW00012916,STATION_USW00013960,STATION_USW00024025,STATION_USW00024233,STATION_USW00093067,STATION_USW00093225,STATION_USW00094728
0,2010-01-16,15.6,15.0,-1.1,13.700000,3.251154,10.0,16.1,15.0,12.98,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2010-01-17,14.4,15.6,0.6,15.566667,0.550757,15.0,16.1,15.6,14.22,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2010-01-18,17.2,14.4,-1.2,15.000000,0.600000,14.4,15.6,15.0,14.22,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2010-01-19,21.1,17.2,2.8,15.733333,1.404754,14.4,17.2,15.6,15.66,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2010-01-20,23.9,21.1,3.9,17.566667,3.365016,14.4,21.1,17.2,16.66,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


In [4]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(scaler.transform(X_train), columns=scaler.feature_names_in_)
X_valid = pd.DataFrame(scaler.transform(X_valid), columns=scaler.feature_names_in_)
X_test = pd.DataFrame(scaler.transform(X_test), columns=scaler.feature_names_in_)

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# LinearSVR

In [6]:
model = LinearSVR(max_iter=10000, random_state=rng)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.8801153467853586
Valid MAE: 3.09208354630214


In [7]:
param_grid = {
    'loss': ['epsilon_insensitive', 'squared_epsilon_insensitive'],
    'C': [3, 2.5, 2.25, 2, 1.75, 1.5, 1.25, 1, 0.75, 0.5, 0.25]
}

model = LinearSVR(max_iter=10000, random_state=rng)
grid = GridSearchCV(
    model, param_grid, cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=2
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

model = LinearSVR(**grid.best_params_, max_iter=100000, random_state=rng)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Fitting 5 folds for each of 22 candidates, totalling 110 fits
[CV] END ......................C=3, loss=epsilon_insensitive; total time=   6.1s
[CV] END ......................C=3, loss=epsilon_insensitive; total time=   6.6s
[CV] END ......................C=3, loss=epsilon_insensitive; total time=   6.4s
[CV] END ......................C=3, loss=epsilon_insensitive; total time=   6.2s
[CV] END ......................C=3, loss=epsilon_insensitive; total time=   5.6s
[CV] END ..............C=3, loss=squared_epsilon_insensitive; total time= 2.3min
[CV] END ..............C=3, loss=squared_epsilon_insensitive; total time= 2.3min
[CV] END ..............C=3, loss=squared_epsilon_insensitive; total time= 2.3min
[CV] END ..............C=3, loss=squared_epsilon_insensitive; total time= 1.9min
[CV] END ..............C=3, loss=squared_epsilon_insensitive; total time= 2.1min
[CV] END ....................C=2.5, loss=epsilon_insensitive; total time=   5.4s
[CV] END ....................C=2.5, loss=epsilo

In [9]:
param_grid = {
    'loss': ['epsilon_insensitive'],
    'C': [0.5, 0.4, 0.3, 0.25, 0.15, 0.1, 0.05, 0.01, 0.005, 0.001]
}

model = LinearSVR(max_iter=10000, random_state=rng)
grid = GridSearchCV(
    model, param_grid, cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=2
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

model = LinearSVR(**grid.best_params_, max_iter=100000, random_state=rng)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ....................C=0.5, loss=epsilon_insensitive; total time=   1.2s
[CV] END ....................C=0.5, loss=epsilon_insensitive; total time=   1.3s
[CV] END ....................C=0.5, loss=epsilon_insensitive; total time=   1.4s
[CV] END ....................C=0.5, loss=epsilon_insensitive; total time=   1.3s
[CV] END ....................C=0.5, loss=epsilon_insensitive; total time=   1.2s
[CV] END ....................C=0.4, loss=epsilon_insensitive; total time=   1.2s
[CV] END ....................C=0.4, loss=epsilon_insensitive; total time=   1.2s
[CV] END ....................C=0.4, loss=epsilon_insensitive; total time=   1.1s
[CV] END ....................C=0.4, loss=epsilon_insensitive; total time=   1.1s
[CV] END ....................C=0.4, loss=epsilon_insensitive; total time=   1.1s
[CV] END ....................C=0.3, loss=epsilon_insensitive; total time=   0.9s
[CV] END ....................C=0.3, loss=epsilon

# SVR

In [5]:
param_grid = [
    {'kernel': ['sigmoid'],
     'coef0': [0, 0.5, 1],
     'gamma': ['scale', 'auto'],
     'C': [0.1, 0.3, 0.4, 0.5, 0.75, 1]},
    {'kernel': ['rbf'],
     'gamma': ['scale', 'auto'],
     'C': [0.1, 0.3, 0.4, 0.5, 0.75, 1]}
]

model = SVR(max_iter=10000)
grid = GridSearchCV(
    model, param_grid, cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=2
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

model = SVR(**grid.best_params_, max_iter=100000)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Fitting 3 folds for each of 48 candidates, totalling 144 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ........C=0.1, coef0=0, gamma=scale, kernel=sigmoid; total time=  51.4s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ........C=0.1, coef0=0, gamma=scale, kernel=sigmoid; total time=  51.7s


...

[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=  50.2s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ........................C=1, gamma=auto, kernel=rbf; total time=  51.3s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Best score:  -3.301047954439653
Best params:  {'C': 0.75, 'gamma': 'auto', 'kernel': 'rbf'}
Train MAE: 2.7927443158232523
Valid MAE: 3.119961395564631


In [8]:
model = SVR(kernel='rbf', C=0.75, gamma='auto', max_iter=100000)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Train MAE: 2.7927443158232523
Valid MAE: 3.119961395564631


# SVR - polynomial kernel

In [5]:
param_grid = [
    {'kernel': ['poly'],
     'degree': [3, 2],
     'coef0': [0, 0.5, 1],
     'gamma': ['scale', 'auto'],
     'C': [0.1, 0.3, 0.4, 0.5, 0.75, 1]},
]

model = SVR(max_iter=10000)
grid = GridSearchCV(
    model, param_grid, cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=2
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

model = SVR(**grid.best_params_, max_iter=100000)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Fitting 3 folds for each of 72 candidates, totalling 216 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END .C=0.1, coef0=0, degree=3, gamma=scale, kernel=poly; total time=  39.9s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END .C=0.1, coef0=0, degree=3, gamma=scale, kernel=poly; total time=  37.9s


...

[CV] END ....C=1, coef0=1, degree=2, gamma=auto, kernel=poly; total time=  37.3s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END ....C=1, coef0=1, degree=2, gamma=auto, kernel=poly; total time=  37.7s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Best score:  -3.1049329430464163
Best params:  {'C': 0.75, 'coef0': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
Train MAE: 2.8179875951554556
Valid MAE: 3.0528539464198268


In [6]:
param_grid = [
    {'kernel': ['poly'],
     'degree': [2],
     'coef0': [0, 1, 5, 10, 15, 20, 25, 30],
     'gamma': ['scale'],
     'C': [0.75]},
]

model = SVR(max_iter=10000)
grid = GridSearchCV(
    model, param_grid, cv=3,
    scoring='neg_mean_absolute_error',
    n_jobs=1, verbose=2
)
grid.fit(X_train, y_train)
print('Best score: ', grid.best_score_)
print('Best params: ', grid.best_params_)

model = SVR(**grid.best_params_, max_iter=100000)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Fitting 3 folds for each of 8 candidates, totalling 24 fits


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=0.75, coef0=0, degree=2, gamma=scale, kernel=poly; total time=  37.2s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=0.75, coef0=0, degree=2, gamma=scale, kernel=poly; total time=  38.1s


...

[CV] END C=0.75, coef0=30, degree=2, gamma=scale, kernel=poly; total time=  30.2s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


[CV] END C=0.75, coef0=30, degree=2, gamma=scale, kernel=poly; total time=  29.5s


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Best score:  -3.1049329430464163
Best params:  {'C': 0.75, 'coef0': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'poly'}
Train MAE: 2.8179875951554556
Valid MAE: 3.0528539464198268
